In [264]:
import pandas

df_train = pandas.read_csv('./50_dataset.csv', quotechar="\'")
df_test = pandas.read_csv('./test.csv', quotechar="\'")

In [265]:
import re
import string
import demoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer

demoji.download_codes()
def clean_tweets(data):
    new_data = data.copy()

    username_hash = r'[#@]\w+'
    punctuation = '[%s]+' % re.escape(string.punctuation)
    special_char = r'[^0-9a-zA-Z\s]+'
    number = r'[0-9]+'
    space = r'\s{2,}'
    space_begin_end = r'^\s+|\s+$'
    url = r'(https?|www):\/{1,}\w+\W+\w+\/{1,}\w+'
    char_ref = r'&\w+;'

    for i in range(len(new_data)):
        new_data[i] = re.sub(char_ref, ' ', new_data[i])
        new_data[i] = re.sub(username_hash, '', new_data[i])
        new_data[i] = re.sub(url, '', new_data[i])
        new_data[i] = re.sub(punctuation, '', new_data[i])
        new_data[i] = re.sub(number, '', new_data[i])
        new_data[i] = re.sub(space_begin_end, '', new_data[i])
        new_data[i] = re.sub(space, '', new_data[i])
        new_data[i] = demoji.replace(new_data[i], '')
        new_data[i] = re.sub(special_char, '', new_data[i])

    return new_data

def case_fold(data):
    new_data = data.copy()
    return list(map(lambda s: s.lower(), new_data))

def tokenize(data):
    new_data = data.copy()
    return list(map(lambda s: s.split(' '), new_data))

def stem(data):
    new_data = data.copy()
    stemmer = StemmerFactory().create_stemmer()

    return list(map(lambda s: stemmer.stem(s), new_data))

def get_term(data):
    new_data = data.copy()
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(new_data)

    return count_vectorizer.get_feature_names()

... OK (Got response in 0.44 seconds)
Writing emoji data to /Users/bangkodir/.demoji/codes.json ...
... OK


In [266]:
tweet_train = stem(case_fold(clean_tweets(df_train['tweet'])))

In [267]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(tweet_train, df_train['category'], random_state=13)

enc = LabelEncoder()
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

print(enc.classes_)
print(y_train)
print(y_test)

['negatif' 'positif']
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 1 0 0 0 1 0]


In [268]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
vect.fit(tweets)

X_train_tfidf = vect.transform(X_train)
X_test_tfidf = vect.transform(X_test)

print(vect.vocabulary_)

{'said': 392, 'iqbal': 168, 'kspi': 230, 'akan': 9, 'aksi': 10, 'besarbesaran': 62, 'desak': 106, 'dpr': 114, 'uji': 476, 'ulang': 477, 'omnibus': 329, 'law': 248, 'sampai': 398, 'hari': 142, 'ini': 161, 'kita': 214, 'bingung': 67, 'karena': 195, 'draf': 117, 'tidak': 457, 'buka': 76, 'mana': 274, 'yang': 496, 'benar': 55, 'adaversi': 3, 'bayang': 50, 'itu': 170, 'kata': 197, 'feri': 126, 'amsari': 17, 'banyak': 44, 'berita': 60, 'kutip': 235, 'polisi': 367, 'bahwa': 38, 'tangkap': 443, 'adalah': 1, 'anggur': 22, 'mungkin': 306, 'mau': 283, 'kesan': 211, 'tolak': 462, 'bukan': 77, 'siapasiapa': 420, 'padahal': 336, 'harus': 143, 'baca': 34, 'begini': 52, 'saja': 393, 'percaya': 353, 'uu': 485, 'cipta': 89, 'kerja': 207, 'untuk': 480, 'mereka': 291, 'juta': 185, 'rakyat': 376, 'indonesia': 158, 'yg': 497, 'nonton': 320, 'mata': 282, 'najwa': 309, 'malam': 272, 'jadi': 171, 'saksi': 394, 'anggota': 21, 'dari': 97, 'pdip': 350, 'kena': 205, 'proses': 372, 'lahir': 240, 'kalau': 188, 'ada'

In [269]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_c = SVC(kernel='linear')
svm_c.fit(X_train_tfidf, y_train)
pred = svm_c.predict(X_test_tfidf)

print(accuracy_score(pred, y_test))

report = classification_report(y_test, pred, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['0'])

0.7692307692307693
positive:  {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}
negative:  {'precision': 0.7692307692307693, 'recall': 1.0, 'f1-score': 0.8695652173913044, 'support': 10}
